In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Now Import Some Important Libs for this Project

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import plotly.express as px


from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.metrics import f1_score, confusion_matrix, precision_recall_curve, roc_curve
import lightgbm as lgb
from lightgbm import plot_importance
import xgboost as xgb



plt.style.use('seaborn')

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [ ]:
df_train = pd.read_csv('/kaggle/input/santander-customer-transaction-prediction/train.csv')
df_test = pd.read_csv('/kaggle/input/santander-customer-transaction-prediction/test.csv')

# Now See head and tail of the dataset

In [ ]:
df_train.head(10)

In [ ]:
df_train.tail()

In [ ]:
df_test.head()

In [ ]:
df_test.tail()

# Now Some EDA Part

In [ ]:
col_features = df_train.drop(['ID_code','target'],axis=1)

In [ ]:
df_train.shape

In [ ]:
df_train.describe()

In [ ]:
df_train.info()

# Now Some Visualization 

In [ ]:
f, ax = plt.subplots(1,2,figsize=(10,4))

df_train['target'].value_counts().plot.pie(
    explode=[0,0.1],autopct='%1.1f%%',ax=ax[0],shadow=True
)

sns.countplot('target', data=df_train, ax=ax[1])

plt.show()

# Now Check Null Values

In [ ]:
null_value = df_train.isnull().sum().sort_values(ascending = False)
null_percent = round(df_train.isnull().sum().sort_values(ascending = False)/len(df_train)*100, 2)

pd.concat([null_value,null_percent], axis=1,keys=['Null values','Percent'])

In [ ]:
plt.figure(figsize=(16,6))
plt.title("Distribution of mean values per row in the train and test set")

sns.distplot(df_train[col_features.columns].mean(axis=1),color="black", kde=True,bins=120, label='train')
sns.distplot(df_test[col_features.columns].mean(axis=1),color="red", kde=True,bins=120, label='test')

plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(16,6))
plt.title("Distribution of mean values per column in the train and test set")

sns.distplot(df_train[col_features.columns].mean(),color="black", kde=True,bins=120, label='train')
sns.distplot(df_test[col_features.columns].mean(),color="red", kde=True,bins=120, label='test')

plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(16,6))
plt.title("Distribution of std values per rows in the train and test set")

sns.distplot(df_train[col_features.columns].std(axis=1),color="blue",kde=True,bins=120, label='train')
sns.distplot(df_test[col_features.columns].std(axis=1),color="green", kde=True,bins=120, label='test')

plt.legend()
plt.show()

In [ ]:
train_df_0 = df_train[df_train['target'] == 0]
train_df_1 = df_train[df_train['target'] == 1]

plt.figure(figsize=(16,6))
plt.title("Distribution of skew values per row in the train set")

sns.distplot(train_df_0[col_features.columns].skew(axis=1),color="red", kde=True,bins=120, label='target = 0')
sns.distplot(train_df_1[col_features.columns].skew(axis=1),color="blue", kde=True,bins=120, label='target = 1')

plt.legend()
plt.show()

In [ ]:
train_df_0 = df_train[df_train['target'] == 0]
train_df_1 = df_train[df_train['target'] == 1]

plt.figure(figsize=(16,6))
plt.title("Distribution of skew values per column in the train set")

sns.distplot(train_df_0[col_features.columns].skew(),color="red", kde=True,bins=120, label='target = 0')
sns.distplot(train_df_1[col_features.columns].skew(),color="blue", kde=True,bins=120, label='target = 1')

plt.legend()
plt.show()

# Now Preprocessing Part

In [ ]:
df_train.drop('ID_code', axis=1, inplace=True)
df_test.drop('ID_code', axis=1, inplace=True)

In [ ]:
df_train.head(10)

# Now Feature Egineering

In [ ]:
X = df_train.drop(['target'], axis=1)
y = df_train['target']

# Now Standard Scaler the data

In [ ]:
scaler = StandardScaler()
X_scaler = scaler.fit_transform(X)
X_scaler_df = pd.DataFrame(X_scaler, columns=X.columns)

In [ ]:
pca = PCA(n_components=2)
x_scaler_pca = pca.fit_transform(X_scaler)
x_scaler_pca_df = pd.DataFrame(x_scaler_pca)

# Now Test Data Standard Scaler the data

In [ ]:
test_scaler = StandardScaler()
trans_test_scaler = test_scaler.fit_transform(df_test)
trans_test_scaler_df = pd.DataFrame(trans_test_scaler, columns=df_test.columns)

In [ ]:
x_scaler_pca_df.head()

In [ ]:
pca.explained_variance_ratio_

In [ ]:
sum(pca.explained_variance_ratio_)

In [ ]:
x_scaler_pca_df['target'] = y

In [ ]:
plt.scatter(x_scaler_pca_df.loc[:, 0], x_scaler_pca_df.loc[:, 1], c=y,  cmap="copper_r")

plt.axis('off')

plt.colorbar()

plt.show()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=0)

In [ ]:
print("Train Dataset shape {} / {}".format(x_train.shape, y_train.shape))
print("Test Dataset shape {} / {}".format(x_test.shape, y_test.shape))

# Now Part Modeling

In [ ]:
train_data = lgb.Dataset(x_train, label=y_train)
val_data = lgb.Dataset(x_test, label=y_test)
params = {
    'n_estimators': 5000,
    'num_leaves': 20,
    'max_depth': -1,
    'min_data_in_leaf': 80,
    'learning_rate': 0.01,
    'boosting': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'Is_training_metric': True,
    'n_jobs': -1
}

In [ ]:
model = lgb.train(params,
                  train_data,
                  valid_sets=val_data, 
                  valid_names=['train','valid'],
                  early_stopping_rounds=100)

# Now Submission Part

In [ ]:
submission = pd.read_csv('/kaggle/input/santander-customer-transaction-prediction/sample_submission.csv')

In [ ]:
submission.head()

# Now Predict

In [ ]:
target = model.predict(df_test)

In [ ]:
submission['target'] = target

In [ ]:
submission.head()

In [ ]:
submission.to_csv('submission.csv', index=False)

In [ ]:
scaler_submission = pd.read_csv('/kaggle/input/santander-customer-transaction-prediction/sample_submission.csv')

In [ ]:
scaler_submission.head()

In [ ]:
scaler_target = model.predict(trans_test_scaler_df)

In [ ]:
scaler_submission['target'] = scaler_target

In [ ]:
scaler_submission.to_csv('scaler_submission.csv', index=False)